### Exploration Platform

In [1]:
from utils import Platform
from utils.training import swarm

# Optional: Decentralised shuffle seed

config = {
    "data":{
        "path": "/mnt/ZOD",
        "version":"full",
        "ratio":0.15,
        "img_size": [400, 1420],
        "transforms":"[CustomCrop(800, 500, 800, 2840), Resize((400, 1420))]",
        "shuffle_seed": 42,
        "decentralised_shuffle_seed": 42,
        "dataloader_args": {
            "prefetch_factor": 2,
            "num_workers": 2,
            "batch_size": 32,
        },
        "train_val_id_generator":"balanced_frames"
    },
    "model":{
        "name": "default",
        "args": {
            "num_output":66
        },
        "loss":"MSELoss"
    },
    "central":{
        "train":"true",
        "use_gpu":"true",
        "epochs": 15
    },
    "decentralised":{
        "train": [],
        "global":{
            "n_clients":5,
            "global_rounds":3,
            "client_resources":{
                "num_cpus": 2, 
                "num_gpus": 0.2
            },
            "ray_init_args":{
                "include_dashboard": True,
                "num_cpus": 12,
                "num_gpus": 2
            },
            "swarm_orchestrator": "synchronous_fixed_rounds_fc",
        },
        "client": {
            "epochs": 10
        }
    },
    "note": "ROI Test"
}

platform = Platform(config)

/home/alex/miniconda3/envs/swarm/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading val frames: 100%|██████████| 10024/10024 [00:01<00:00, 9120.87it/s]


# Other things

In [ ]:
from zod.visualization.oxts_on_image import visualize_oxts_on_image
from zod.constants import Camera, Lidar, Anonymization

# extract the oxts data
oxts = zod_frame.oxts

# visualize the oxts data on the image
calibrations = zod_frame.calibration
from tbparse import SummaryReader
log_dir = "runs"
reader = SummaryReader(log_dir, extra_columns=set(["dir_name"]))
df = reader.scalars


global_metrics = dict(
    centralised = "centralised/loss/epoch",
    federated = "federated/loss/global",
)

reader.scalars.loc[df['tag'].isin(global_metrics.values())].info.keyframe_time.timestamp()

image = zod_frame.get_image(Anonymization.DNAT)
image = visualize_oxts_on_image(oxts, key_timestamp, calibrations, image, camera=Camera.FRONT)

plt.axis("off")
plt.imshow(image)
plt.show()

In [ ]:
from tbparse import SummaryReader
import plotly.express as px
import matplotlib.pyplot as plt

def get_name(s):
    name_chunks = s.split("/")[1].split("_")
    name = name_chunks[0] + " " + name_chunks[-1]
    return name

def get_run(s):
    return s.split("/")[0]

log_dir = "runs"
reader = SummaryReader(log_dir, extra_columns=set(["dir_name"]))
df = reader.scalars


global_metrics = dict(
    centralised = "centralised/loss/epoch",
    federated = "federated/loss",
)

global_results = reader.scalars.loc[df['tag'].isin(global_metrics.values())]

global_results["dir_name"]

# Transformations
global_results["Methods/Sets"] = global_results["dir_name"].apply(get_name)
global_results["Run"] = global_results["dir_name"].apply(get_run)

fig = px.line(
    global_results, 
    x="step", 
    y="value", 
    color="Methods/Sets", 
    width=1280, 
    height=720,
    facet_row="Run")
fig.update_layout(
    template="ggplot2",
    font=dict(family="Helvetica", size=12, color="#404040"),
    title=dict(text="Benchmarks", font=dict(size=24)),
    xaxis=dict(title="Step", showgrid=True),
    yaxis=dict(title="Loss", showgrid=True),
)
fig.show()